# Export-Script
<p>This Export Script uses the Google Apis to first download to a specific folder and the delete the files from google drive</p>

In [ ]:
#Enable Google Drive
#needs credentials.json if used for the first use or token.json if used for recurrent uses
#import libraries
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json',['https://www.googleapis.com/auth/drive'])
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', ['https://www.googleapis.com/auth/drive'])
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('drive', 'v3', credentials=creds)

In [ ]:
def download_file(file_id, filename):
    """ this function is responsible for the download process of the script.

    Args:
        file_id (str): Unique Identifier set by Google to identify every File in Drive
        filename (str): Filename of the File in Google Drive. The file is saved with the same filename
    """
    request = service.files().get_media(fileId=file_id)
    fh = io.FileIO(filename, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print('Download done')

In [ ]:
import io
from googleapiclient.http import MediaIoBaseDownload
# Call the Drive v3 API
dirpath = '../data'
os.makedirs(dirpath,exist_ok=True)
#Let the Script run as long as there are less Files downloaded than in total
while len([name for name in os.listdir(dirpath)]) != int(input("Länge des Dataframes: ")):
    #Get every File, which is an image with the fileending "tiff"
    results = service.files().list(q="mimeType='image/tiff'",spaces='drive',
                                          fields='nextPageToken, files(id, name)',
                                          pageToken=None).execute()
    items = results.get('files', [])
    #iterate over the names to download each one
    for item in items:
        #get the Unique Identifier from Google
        file_id = item.get('id')
        #get the Filename of the Item to locally save it with the same name
        filename = item.get('name')
        print("Download " + str(filename))
        #call the function to download the file and save it
        download_file(item['id'], item['name'])
        #Delete the File from Google Drive in total(Files in the Drive Trash still takes up space, so it is necessary to completely delete them)
        service.files().delete(fileId=file_id).execute()
        #because the files are in the wrong directory, we move them to the correct one
        os.replace(filename,dirpath + "/" + filename)
        print(str(len([name for name in os.listdir(dirpath)]))+ "/" + str(len(df)))